# Data info
- [Dacon] 고객 대출등급 분류
- https://dacon.io/competitions/official/236214/overview/description

- 5개 팀의 코드를 참고하여 분석 진행해보기

# Data load

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv("./data/credit_rating_train.csv")
test = pd.read_csv("./data/credit_rating_test.csv")
sample_submission = pd.read_csv("./data/credit_rating_sample_submission.csv")

In [3]:
train.head()

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0,0,0,C
1,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060,0,0,B
2,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944,0,0,A
3,TRAIN_00003,14400000,36 months,8 years,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108,0,0,C
4,TRAIN_00004,18000000,60 months,Unknown,RENT,71736000,25.39,19,주요 구매,0,228540,148956,0,0,B


In [4]:
test.head()

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
0,TEST_00000,16800000,36 months,8 years,MORTGAGE,132000000,19.64,12,주택 개선,0,394692,146604,0,0
1,TEST_00001,8400000,36 months,5 years,RENT,89971200,15.84,25,부채 통합,0,0,0,0,0
2,TEST_00002,17280000,36 months,6 years,RENT,150000000,8.41,20,신용 카드,0,1786980,281820,0,0
3,TEST_00003,14400000,36 months,5 years,MORTGAGE,66000000,13.72,30,신용 카드,1,669024,281724,0,0
4,TEST_00004,27600000,36 months,5 years,RENT,55200000,30.50,12,신용 카드,0,1250052,614844,0,0


In [5]:
sample_submission.head()

,ID,대출등급
0,TEST_00000,G
1,TEST_00001,G
2,TEST_00002,G
3,TEST_00003,G
4,TEST_00004,G


In [6]:
print(train.shape)
print(test.shape)
print(sample_submission.shape)

(96294, 15)
(64197, 14)
(64197, 2)


# EDA

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96294 entries, 0 to 96293
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            96294 non-null  object 
 1   대출금액          96294 non-null  int64  
 2   대출기간          96294 non-null  object 
 3   근로기간          96294 non-null  object 
 4   주택소유상태        96294 non-null  object 
 5   연간소득          96294 non-null  int64  
 6   부채_대비_소득_비율   96294 non-null  float64
 7   총계좌수          96294 non-null  int64  
 8   대출목적          96294 non-null  object 
 9   최근_2년간_연체_횟수  96294 non-null  int64  
 10  총상환원금         96294 non-null  int64  
 11  총상환이자         96294 non-null  int64  
 12  총연체금액         96294 non-null  int64  
 13  연체계좌수         96294 non-null  int64  
 14  대출등급          96294 non-null  object 
dtypes: float64(1), int64(8), object(6)
memory usage: 11.0+ MB


In [9]:
train.isnull().sum()

ID              0
대출금액            0
대출기간            0
근로기간            0
주택소유상태          0
연간소득            0
부채_대비_소득_비율     0
총계좌수            0
대출목적            0
최근_2년간_연체_횟수    0
총상환원금           0
총상환이자           0
총연체금액           0
연체계좌수           0
대출등급            0
dtype: int64

- 데이터 상에는 결측치 없음.
- 그러나, head로 확인했을 때 근로기간에 Unknown 값이 있었으므로 결측치라고 볼 수도 있을 것.

## 근로기간

In [10]:
train["근로기간"].value_counts()

근로기간
10+ years    31585
2 years       8450
< 1 year      7774
3 years       7581
1 year        6249
Unknown       5671
5 years       5665
4 years       5588
8 years       4888
6 years       3874
7 years       3814
9 years       3744
10+years       896
<1 year        370
3               89
1 years         56
Name: count, dtype: int64

- 띄어쓰기 등의 차이로 중복값 존재

## 범주형 변수 확인

In [7]:
train.nunique()

ID              96294
대출금액             1340
대출기간                2
근로기간               16
주택소유상태              4
연간소득             7726
부채_대비_소득_비율      4025
총계좌수              109
대출목적               12
최근_2년간_연체_횟수       20
총상환원금           27797
총상환이자           35626
총연체금액             109
연체계좌수               4
대출등급                7
dtype: int64

In [8]:
test.nunique()

ID              64197
대출금액             1309
대출기간                2
근로기간               16
주택소유상태              3
연간소득             5842
부채_대비_소득_비율      3992
총계좌수              105
대출목적               13
최근_2년간_연체_횟수       16
총상환원금           21547
총상환이자           27902
총연체금액              71
연체계좌수               5
dtype: int64

- 범주가 20개 이하인 컬럼만 확인해본다.

In [17]:
# train
nuniq = train.nunique()

for col in nuniq[nuniq < 21].index:
    
    print(train[col].value_counts(), "\n")

대출기간
36 months    64479
60 months    31815
Name: count, dtype: int64 

근로기간
10+ years    31585
2 years       8450
< 1 year      7774
3 years       7581
1 year        6249
Unknown       5671
5 years       5665
4 years       5588
8 years       4888
6 years       3874
7 years       3814
9 years       3744
10+years       896
<1 year        370
3               89
1 years         56
Name: count, dtype: int64 

주택소유상태
MORTGAGE    47934
RENT        37705
OWN         10654
ANY             1
Name: count, dtype: int64 

대출목적
부채 통합     55150
신용 카드     24500
주택 개선      6160
기타         4725
주요 구매      1803
의료         1039
자동차         797
소규모 사업      787
이사          506
휴가          466
주택          301
재생 에너지       60
Name: count, dtype: int64 

최근_2년간_연체_횟수
0     76392
1     13079
2      3896
3      1430
4       697
5       338
6       178
7        86
8        67
9        40
10       35
11       21
12        9
14        8
13        8
16        4
18        2
15        2
17        1
30        1
Name: c

In [16]:
# test
nuniq = test.nunique()

for col in nuniq[nuniq < 21].index:
    print(test[col].value_counts(), "\n")

대출기간
36 months    43229
60 months    20968
Name: count, dtype: int64 

근로기간
10+ years    20937
2 years       5609
< 1 year      5143
3 years       5036
1 year        4235
5 years       3927
Unknown       3862
4 years       3662
8 years       3284
9 years       2545
7 years       2515
6 years       2479
10+years       616
<1 year        259
3               49
1 years         39
Name: count, dtype: int64 

주택소유상태
MORTGAGE    31739
RENT        25281
OWN          7177
Name: count, dtype: int64 

대출목적
부채 통합     37054
신용 카드     16204
주택 개선      4019
기타         3032
주요 구매      1244
의료          696
소규모 사업      541
자동차         536
이사          362
휴가          294
주택          185
재생 에너지       29
결혼            1
Name: count, dtype: int64 

최근_2년간_연체_횟수
0     50966
1      8703
2      2594
3       968
4       436
5       218
6       125
7        72
8        35
9        24
10       22
11       14
13        9
12        7
15        2
14        2
Name: count, dtype: int64 

연체계좌수
0    63869
1      307
2

### 주택소유상태 - "ANY" 확인

In [9]:
train["주택소유상태"].value_counts()

주택소유상태
MORTGAGE    47934
RENT        37705
OWN         10654
ANY             1
Name: count, dtype: int64

In [10]:
test["주택소유상태"].value_counts()

주택소유상태
MORTGAGE    31739
RENT        25281
OWN          7177
Name: count, dtype: int64

- test 데이터에도 없고, train 데이터에서도 1개뿐이므로 삭제해준다.

In [23]:
train = train[train["주택소유상태"]!="ANY"]

# 확인
train["주택소유상태"].unique()

array(['RENT', 'MORTGAGE', 'OWN'], dtype=object)

### 대출기간 확인

In [11]:
train["대출기간"].unique()

array(['36 months', '60 months'], dtype=object)

- 대출 시 선택지가 2가지뿐인듯